<a href="https://colab.research.google.com/github/ngohieu12/NgauNhien/blob/master/Python/ChuongTrinh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


class Algorithms:
    def __init__(self, delta=100, epsilon=0.0001):
        x = np.loadtxt('m_n_alpha.txt')
        m = int(x[0])
        n = int(x[1])
        alpha = x[2]
        p = []
        f = []
        x = 'p{}.txt'
        y = 'f{}.txt'
        for i in range(m):
            p.append(np.loadtxt(x.format(i)))
            f.append(np.loadtxt(y.format(i)))
        self.p = p
        self.m = m
        self.n = n
        self.f = f
        self.alpha = alpha
        self.delta = delta
        self.epsilon = epsilon

    def ImprovementAlgorithm(self):
        m = self.m
        n = self.n
        alpha = self.alpha
        p = self.p
        f = self.f
        v = np.zeros(n)
        delta = self.delta
        epsilon = self.epsilon
        listV = []
        while delta > epsilon:
            q = np.zeros((m, n))
            for i in range(m):
                q[i] = f[i] + alpha * p[i].dot(v)
            w = q[0]
            a = np.ones(n, dtype=np.int64)

            for i in range(n):
                for j in range(m):
                    if q[j][i] < w[i]:
                        w[i] = q[j][i]
                        a[i] = j + 1
            delta = max(abs(v - w))
            v = w
            listV.append(v)
         

        for i in range(m):
            q[i] = f[i] + alpha * p[i].dot(v)
        w = q[0]
        a = np.zeros(n, dtype=np.int64)
        for i in range(n):
            for j in range(m):
                if q[j][i] < w[i]:
                    w[i] = q[j][i]
                    a[i] = j
                    
                    
        result = dict()
        result['v'] = v
        result['a'] = a+1
        result['listV'] = listV
        return result

    def PolicyImprovementAlgorithm(self):
        m = self.m
        n = self.n
        alpha = self.alpha
        p = self.p
        f = self.f
        v = np.zeros(n)
        delta = self.delta
        epsilon = self.epsilon
        w = f[0]
        a = np.zeros(n, dtype=np.int64)
        g = np.zeros(n)
        I = np.eye(n)
        for i in range(n):
            for j in range(m):
                if f[j][i] < w[i]:
                    w[i] = f[j][i]
                    a[i] = j

        number = 0
        listA=[]
        listV=[]
        while (delta > epsilon):
            number = number + 1
            listA.append(a+1)

            for i in range(n):
                g[i] = f[a[i]][i]

            q = np.zeros((n, n))
            
            for i in range(n):
                q[i] = p[a[i]][i]

            v = np.linalg.inv(I - alpha * q).dot(g)
            listV.append(v)

            k = np.zeros((m, n))
            for i in range(m):
                k[i] = f[i] + alpha * p[i].dot(v)
            w = k[0]

            b = np.zeros(n, dtype=np.int64)
            for i in range(n):
                for j in range(m):
                    if k[j][i] < w[i]:
                        w[i] = k[j][i]
                        b[i] = j

            delta = max(abs(a - b))
            a = b
            
        result = dict()
        result['listA'] = listA
        result['listV'] = listV
        result['number']= number
        return result

    def PolicyImprovementforAverageCosts(self):
        m = self.m
        n = self.n
        p = self.p
        f = self.f
        delta = self.delta
        epsilon = self.epsilon
        A = np.eye(n + 1)
        A[n][0] = 1
        A[n][n] = 0
        for i in range(n):
            A[i][n] = 1

        w = f[0]
        a = np.zeros(n, dtype=np.int64)
        g = np.zeros(n + 1)
        h = np.zeros(n)

        listA=[]
        listH=[]
        listPhi=[]

        for i in range(n):
            for j in range(m):
                if f[j][i] < w[i]:
                    w[i] = f[j][i]
                    a[i] = j

        number = 0
        while (delta > epsilon):
            number = number + 1
            listA.append(a+1)

            for i in range(n):
                g[i] = f[a[i]][i]
            g[n] = 0

            q = np.zeros((n + 1, n + 1))
            for i in range(n):
                for j in range(n):
                    q[i][j] = p[a[i]][i][j]
            for i in range(n):
                q[i][n] = q[n][i] = 0

            v = np.linalg.inv(A - q).dot(g)
            

            for i in range(n):
                h[i] = v[i]
                
            listH.append(h)
            listPhi.append(v[n])

            k = np.zeros((m, n))
            for i in range(m):
                k[i] = f[i] + p[i].dot(h)
            w = k[0]

            b = np.zeros(n, dtype=np.int64)
            for i in range(n):
                for j in range(m):
                    if k[j][i] < w[i]:
                        w[i] = k[j][i]
                        b[i] = j

            delta = max(abs(a - b))
            a = b
            
        result = dict()
        result['listA'] = listA
        result['listH'] = listH
        result['listPhi']=listPhi
        result['number']= number
        return result



def mainMenu ():
    print("********************* Quá trình quyết định Markov (MDP) *********************")
    print("1. Cải thiện chi phí có chiết khấu")
    print("2. Cải tiến chính sách theo tiêu chi tổng chi phí có chiết ")
    print("3. Cải tiến chính sách theo tiêu chí chi phí trung bình dài hạn")
    selection = int(input("Lựa chọn: "))
    if selection == 1:
        method1()
    elif selection == 2:
        method2()
    else: 
        method3()
 
  

def method1 ():
    test = Algorithms()
    a = test.ImprovementAlgorithm()
    for i in a['listV']:
       print("v=",i)
    print("a=",a['a'])
 

def method2 ():
    test = Algorithms()
    a=test.PolicyImprovementAlgorithm()
    for i in range(a['number']):
        print("\nBước lặp", i+1)
        print("a=",a['listA'][i])
        print("v=",a['listV'][i])
  

def method3 ():
    test = Algorithms()
    a=test.PolicyImprovementforAverageCosts()
    for i in range(a['number']):
        print("\nBước lặp", i+1)
        print("a=",a['listA'][i])
        print("h=",a['listH'][i])
        print("Phi=",a['listPhi'][i])
 

#main
mainMenu()